Feature --> polynomial transfomation --> Linear regression --> label

### Getting the diffrent number of feature from the input 

In [3]:
import itertools
import functools

def get_combinations(x,degree):
    return itertools.combinations_with_replacement(x,degree)


def compute_new_features(items):
    return functools.reduce(lambda x,y: x*y,items)

In [4]:
{items: compute_new_features(items) for items in get_combinations([0,1],3)} 

{(0, 0, 0): 0, (0, 0, 1): 0, (0, 1, 1): 0, (1, 1, 1): 1}

### Polynomial Regression

In [7]:
import numpy as np
import functools
import itertools

def polynomail_transform(x,degree,logging=False):
    x = np.array(x)
    if x.ndim == 1:
        x = x[:,None] # If the input is of one dimension then this converts it into a featrue matrix
    x_t = np.transpose(x)
    features = [np.ones(len(x))]
    
    if logging:
        print("Input:",x)
    for degree in range(1,degree+1):
        for items in get_combinations(x_t,degree):
            features.append(compute_new_features(items))
            if logging:
                print(items,':',compute_new_features(items) )
    if logging:
        print(np.asarray(features).transpose())
    return np.asarray(features).transpose()

In [8]:
polynomail_transform([5,4],2,True)

Input: [[5]
 [4]]
(array([5, 4]),) : [5 4]
(array([5, 4]), array([5, 4])) : [25 16]
[[ 1.  5. 25.]
 [ 1.  4. 16.]]


array([[ 1.,  5., 25.],
       [ 1.,  4., 16.]])

In [28]:
polynomail_transform([2,3],3,True)

Input: [[2]
 [3]]
(array([2, 3]),) : [2 3]
(array([2, 3]), array([2, 3])) : [4 9]
(array([2, 3]), array([2, 3]), array([2, 3])) : [ 8 27]
[[ 1.  2.  4.  8.]
 [ 1.  3.  9. 27.]]


array([[ 1.,  2.,  4.,  8.],
       [ 1.,  3.,  9., 27.]])

In [4]:
polynomail_transform([[1],[2],[3],[4],[5]],4,True)

Input: [[1]
 [2]
 [3]
 [4]
 [5]]
(array([1, 2, 3, 4, 5]),) : [1 2 3 4 5]
(array([1, 2, 3, 4, 5]), array([1, 2, 3, 4, 5])) : [ 1  4  9 16 25]
(array([1, 2, 3, 4, 5]), array([1, 2, 3, 4, 5]), array([1, 2, 3, 4, 5])) : [  1   8  27  64 125]
(array([1, 2, 3, 4, 5]), array([1, 2, 3, 4, 5]), array([1, 2, 3, 4, 5]), array([1, 2, 3, 4, 5])) : [  1  16  81 256 625]
[[  1.   1.   1.   1.   1.]
 [  1.   2.   4.   8.  16.]
 [  1.   3.   9.  27.  81.]
 [  1.   4.  16.  64. 256.]
 [  1.   5.  25. 125. 625.]]


array([[  1.,   1.,   1.,   1.,   1.],
       [  1.,   2.,   4.,   8.,  16.],
       [  1.,   3.,   9.,  27.,  81.],
       [  1.,   4.,  16.,  64., 256.],
       [  1.,   5.,  25., 125., 625.]])

### Nonlinear training example

    **y = sin(2pix)**

In [32]:
def create_nonlin_training_data(func,sample_size,std):
    x = np.linspace(0,1,sample_size)
    y = func(x) + np.random.normal(scale=std,size=x.shape)
    
    return x,y

def nonlin(x):
    return np.sin(2*np.pi*x)

### ridge regulariation

In [1]:
from IPython.display import display,Math,Latex
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Loss function for ridge classification :-

**Loss Function**  : $(1/2)(Xw - y)^{T}$$(Xw - y)$ + $(Λ/2)w^{T}w$


**Gradient Calculation** : $X^{T}(Xw - y) + Λw$

**Normal Equation** : $ w = (X^{T}X + ΛI)X^{-1}y$

In [82]:
class LinReg(object):
    def __init__(self,):
        self.t0 = 20
        self.t1 = 100
        
    def predict(self,X:np.ndarray)->np.ndarray:
        y = X @ self.w
        return y
    
    def loss(self,X:np.ndarray,y:np.ndarray,reg_rate:float)-> np.array:
        ## (1/2)*np.transpose(X@w - y)*(X@w - y) --> loss function for linear regression
        ## reg_rate(np.transpose(self.w)*(self.w)) ---> Ridge regression term
        return ((1/2)*np.transpose((X@self.w -y))@(X@self.w -y) - reg_rate*np.transpose(self.w)@self.w)
    
    def rmse(self,X:np.ndarray,y:np.ndarray) -> float:
        return np.sqrt((2/X.shape[0])* self.loss(X,y,0))
    
    
    def fit(self,X:np.ndarray,y:np.ndarray,reg_rate:float) -> float: #Normal equation
        self.w = np.zeros(X.shape[1])
        eye = np.eye(np.size(X,1))
        self.w = np.linalg.solve(
                    reg_rate*eye + np.transpose(X)@X,np.transpose(X)@y
        )
        return self.w
    def calculate_gradient(self,X:np.ndarray,y:np.ndarray,reg_rate:float) -> np.ndarray:
        return np.transpose(X)@(X@self.w-y) + reg_rate*self.w

    def update_weights(self,grad:np.ndarray,lr:float) -> np.ndarray:
        return (self.w - lr*grad)
    
    def learning_schedule(self,t):
        return self.t0/(t+self.t1)

In [83]:
X = np.array([[1,3,2,5],[1,9,4,7]])
w = np.array([[1],[1],[1],[1]])
y =np.array([[6],[11]])
reg_rate = 0.01

In [84]:
y.shape,X.shape,w.shape

((2, 1), (2, 4), (4, 1))

In [85]:
lin_reg = LinReg()
lin_reg.w = w

In [86]:
lin_reg.loss(X,y,reg_rate)

array([[62.46]])

In [87]:
lin_reg.calculate_gradient(X,y,reg_rate)

array([[ 15.01],
       [105.01],
       [ 50.01],
       [ 95.01]])

In [88]:
np.transpose(X)@(X@w-y) + reg_rate*w

array([[ 15.01],
       [105.01],
       [ 50.01],
       [ 95.01]])

### Lasso regression

In [91]:
from sklearn.linear_model import Lasso
reg_rate = 0.01
lasso = Lasso(alpha=reg_rate)
lasso.fit(X,y)

Lasso(alpha=0.01)

### Multipoutput-regression

In [98]:
X.shape,y.shape

((100, 10), (100, 5))

In [102]:
def add_dummy_feature(X):
    return np.column_stack((np.ones(X.shape[0]),X))

In [104]:
X = add_dummy_feature(X)

In [106]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [107]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((80, 11), (20, 11), (80, 5), (20, 5))

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

### Model

$$ Y_{nxk} = X_{nx(m+1)}W_{(m+1)xk} $$

### Loss

$$ J(W) = (1/2)*(Xw - y)^{T}*(Xw - y) $$

### Optimization

$$ 1. Normal Equation$$
$$ 2. Gradient Descent$$

In [118]:
class LinReg(object):
    
    def __init__(self):
        self.t0 = 20
        self.t1 = 100
        
    def predict(self,X:np.ndarray) -> np.ndarray:
        y = X @ self.w
        return y
    def loss(self,X:np.ndarray,y:np.ndarray,reg_rate:float) -> np.ndarray:
        e = y - self.predict(X)
        return (1/2)*np.transpose(e)*(e)
    
    def rmse(self,X:np.ndarray,y:np.ndarray) -> np.ndarray:
        return np.sqrt((2/X.shape[0])* self.loss(X,y,0))
    
    def fit(self,X:np.ndarray,y:np.ndarray,reg_rate:float) -> np.ndarray:
        self.w = np.zeros((X.shape[1],y.shape[1]))
        eye = np.eye(np.size(X,1))
        
        self.w = np.linalg.solve(
            eye*reg_rate + np.transpose(X)@X,np.transpose(X)@y)
        
        return self.w
    def calculate_gradient(self,X:np.ndarray,y:np.ndarray,reg_rate:float) -> np.ndarray:
        e = self.predict(X) - y
        return np.transpose(X)(e) + reg_rate*self.w
    
    def update_weights(self,lr:float,grad:np.ndarray)-> np.ndarray:
        self.w = self.w - lr*grad

In [119]:
lin_reg = LinReg()

In [122]:
w = lin_reg.fit(X_train,y_train,reg_rate=0)

In [123]:
np.testing.assert_almost_equal(w[1:,:],coeff,decimal=2)

In [124]:
w

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [82.34717191, 86.61347063, 96.08125288,  6.51214685,  4.4571111 ],
       [91.32835964, 30.50466984, 55.79874006, 98.2444883 , 40.04485347],
       [99.42330832, 61.47698861,  3.71296039,  1.42515152, 34.21038752],
       [68.638023  , 29.58919764, 30.32919214, 35.58891546, 81.03020815],
       [27.13061013, 25.80592125, 53.23203283, 70.3189016 , 94.92799   ],
       [57.759009  ,  7.52772798,  7.82460997, 37.12869443, 76.65910506],
       [82.19039084, 70.05286228, 88.30775973, 96.65751069, 77.47476142],
       [66.58713983, 40.08795636, 76.81946645, 52.77147256, 23.7523138 ],
       [68.86834265, 70.79823547, 76.72100661, 28.71527129, 54.82562819],
       [69.40873751, 78.1192844 , 16.89261159, 37.4062625 , 41.37802199]])

In [125]:
coeff

array([[82.34717191, 86.61347063, 96.08125288,  6.51214685,  4.4571111 ],
       [91.32835964, 30.50466984, 55.79874006, 98.2444883 , 40.04485347],
       [99.42330832, 61.47698861,  3.71296039,  1.42515152, 34.21038752],
       [68.638023  , 29.58919764, 30.32919214, 35.58891546, 81.03020815],
       [27.13061013, 25.80592125, 53.23203283, 70.3189016 , 94.92799   ],
       [57.759009  ,  7.52772798,  7.82460997, 37.12869443, 76.65910506],
       [82.19039084, 70.05286228, 88.30775973, 96.65751069, 77.47476142],
       [66.58713983, 40.08795636, 76.81946645, 52.77147256, 23.7523138 ],
       [68.86834265, 70.79823547, 76.72100661, 28.71527129, 54.82562819],
       [69.40873751, 78.1192844 , 16.89261159, 37.4062625 , 41.37802199]])

In [131]:
add_x = np.array([0,1]*10)

In [134]:
add_x.shape

(20,)

In [155]:
X = np.array([[1,2,3],[3,2,4]])
y = np.array([[2],[4]])
w = np.array([[0.1],[-0.2],[0.3]])

In [156]:
X.shape,w.shape,y.shape

((2, 3), (3, 1), (2, 1))

In [158]:
lin_reg.w = w
lin_reg.loss(X,y,0)

array([[0.98 , 2.03 ],
       [2.03 , 4.205]])

In [163]:
((1/2)*np.transpose((X@w -y))@(X@w -y)) - 0.1

array([[5.085]])